In [ ]:
def inputMessage():

    ''''
    message = input('message: ')
    encrypt_type = int(input('encyption type (number): '))
    encryption_key = input('encryption key: ')
    encryption_dir = input("encrypt/decrypt")
    '''
    message = 'hello world'
    encrypt_type = 1
    encrypt_key = 7
    encrypt_dir = 'e'

    


In [ ]:
#ef encryptMessage(message, encrypt_type, encrypt_key, encrypt_dir):

    



In [17]:
def ascii_encode(msg):

    lst = []
    for i in msg:
        lst.append(ord(i))

    return lst

#print(ascii_encode('hello world'))

[104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100]


In [18]:
def shift_ascii_numbers(numb_list, shift_val):
    '''
    we will restrict ourselves to the set of single printable characters from the ascii table. These range from 32 to 126 in the ascii table - a total of 95 characters. 
    
    To ensure no overflow after shifting, we first subtract 32 to take us to the range 0 to 94, make the shift modulo 95, then add 32 again to take us back to the suitable range. 
    '''

    shifted_numb_list =  []
    for i in numb_list:
        temp_val = ( (i -32 + shift_val) ) % 95 + 32
        shifted_numb_list.append(temp_val)
    return shifted_numb_list

#lst = shift_ascii_numbers([104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100], 77 )
#lst


[86, 83, 90, 90, 93, 109, 101, 93, 96, 90, 82]

In [19]:
def ascii_decode(msg):
    
    lst = []
    for i in msg:
        lst.append( chr(i))
        
    return lst

#output = ascii_decode([86, 83, 90, 90, 93, 109, 101, 93, 96, 90, 82])
#print(output)
#print(''.join(output))


    
    

['V', 'S', 'Z', 'Z', ']', 'm', 'e', ']', '`', 'Z', 'R']
VSZZ]me]`ZR


In [20]:
def ascii_ceasar_encrypt(message, key):
    
    #step 1: ascii encode
    ascii_message = ascii_encode(message)
    
    #step 2: step 2: shift ascii numbers
    shifted_ascii_message = shift_ascii_numbers(ascii_message, key)
    
    #step 3: ascii decode 
    encrypted_message = ascii_decode(shifted_ascii_message)
    
    #step 4: convert to string and return 
    return ''.join(encrypted_message)

print(ascii_ceasar_encrypt("hello world again", 23))

 |$$'7/'*${7x~x!&


In [21]:
def ascii_ceasar_decrypt(encrypted_message, key):
    
    #step 1: ascii encode
    ascii_message = ascii_encode(encrypted_message)
    
    #step 2: step 2: shift ascii numbers
    shifted_ascii_message = shift_ascii_numbers(ascii_message, -1*key)
    
    #step 3: ascii decode 
    decrypted_message = ascii_decode(shifted_ascii_message)
    
    #step 4: convert to string and return 
    return ''.join(decrypted_message)

In [25]:
#ascii_ceasar_decrypt(" |$$'7/'*${7x~x!&",23)

'gdkkn~vnqkc~`f`hm'

this is a " and this is a '
